# IMPORTS

In [ ]:
!pip install -qq transformers accelerate evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 53.7 MB/s eta 0:00:00


In [ ]:
! pip install -q evaluate rouge_score

  Preparing metadata (setup.py) ... done


In [ ]:
import re
import random
import evaluate
from google.colab import drive
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix, classification_report, accuracy_score, f1_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset
import transformers
from keras.callbacks import EarlyStopping
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, Seq2SeqTrainer, Seq2SeqTrainingArguments, pipeline, DataCollatorForSeq2Seq, T5ForConditionalGeneration, T5Tokenizer
from transformers.data.processors.utils import InputFeatures
from datasets import load_dataset


pd.options.mode.chained_assignment = None # Turn off some annoying pandas warning

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [ ]:
SEED = 42

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Load the dataset


In [ ]:
billsum = load_dataset("billsum", split="ca_test")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/18949 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3269 [00:00<?, ? examples/s]

Generating ca_test split:   0%|          | 0/1237 [00:00<?, ? examples/s]

In [ ]:
billsum

Dataset({
    features: ['text', 'summary', 'title'],
    num_rows: 1237
})

In [ ]:
billsum = billsum.train_test_split(test_size=0.2)

In [ ]:
billsum

DatasetDict({
    train: Dataset({
        features: ['text', 'summary', 'title'],
        num_rows: 989
    })
    test: Dataset({
        features: ['text', 'summary', 'title'],
        num_rows: 248
    })
})

In [ ]:
example = billsum["train"][0]
for key in example:
    print("A key of the example: \"{}\"".format(key))
    print("The value corresponding to the key-\"{}\"\n \"{}\"".format(key, example[key]))

A key of the example: "text"
The value corresponding to the key-"text"
 "The people of the State of California do enact as follows:


SECTION 1.
This act shall be known, and may be cited, as the Pool Safety Act.
SEC. 2.
The Legislature finds and declares all of the following:
(a) Swimming pools provide children and their families with a wonderful opportunity for recreation, exercise, and fun. Keeping children safe during this activity is supported by parents and guardians, safety advocates, health providers, insurance companies, and the swimming pool industry.
(b) According to both the federal Centers for Disease Control and Prevention’s National Center for Injury Prevention and Control and the State Department of Public Health’s EpiCenter data, drowning is the leading cause of death for California children one to four years of age, inclusive.
(c) Additional children suffer near-drowning incidents and survive, but many of those children suffer irreversible brain injuries, which can lea

# Tokenization T5_Model

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("t5-small")

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

In [ ]:
tokenized_text = tokenizer(example['text'])
for key in tokenized_text:
    print(key)
    print(tokenized_text[key])

Token indices sequence length is longer than the specified maximum sequence length for this model (1768 > 512). Running this sequence through the model will result in indexing errors


input_ids
[37, 151, 13, 8, 1015, 13, 1826, 103, 3, 35, 2708, 38, 6963, 10, 180, 3073, 9562, 1300, 100, 1810, 1522, 36, 801, 6, 11, 164, 36, 3, 11675, 6, 38, 8, 9896, 6859, 1983, 5, 180, 3073, 5, 1682, 37, 28204, 12902, 11, 15884, 7, 66, 13, 8, 826, 10, 41, 9, 61, 27223, 14652, 370, 502, 11, 70, 1791, 28, 3, 9, 1627, 1004, 21, 17711, 6, 2510, 6, 11, 694, 5, 3, 18536, 502, 1346, 383, 48, 1756, 19, 3510, 57, 1362, 11, 4879, 7137, 6, 1455, 18408, 6, 533, 3580, 6, 958, 688, 6, 11, 8, 5989, 2201, 681, 5, 41, 115, 61, 2150, 12, 321, 8, 2822, 1166, 7, 21, 14326, 4330, 11, 19715, 22, 7, 868, 1166, 21, 28905, 19715, 11, 4330, 11, 8, 1015, 1775, 13, 2575, 1685, 22, 7, 12741, 24382, 331, 6, 24614, 53, 19, 8, 1374, 1137, 13, 1687, 21, 1826, 502, 80, 12, 662, 203, 13, 1246, 6, 13066, 5, 41, 75, 61, 11180, 502, 5696, 1084, 18, 26, 3623, 29, 53, 15935, 11, 7905, 6, 68, 186, 13, 273, 502, 5696, 19598, 2660, 2317, 2241, 5157, 6, 84, 54, 991, 12, 280, 2961, 1036, 30307, 24, 1113, 59, 163, 8, 4161, 861, 1

# Data Preprocessing

In [ ]:
def preprocess_function(examples):
    # Prepends the string "summarize: " to each document in the 'text' field of the input examples.
    # This is done to instruct the T5 model on the task it needs to perform, which in this case is summarization.
    inputs = ["summarize: " + doc for doc in examples["text"]]

    # Tokenizes the prepended input texts to convert them into a format that can be fed into the T5 model.
    # Sets a maximum token length of 1024, and truncates any text longer than this limit.
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)

    # Tokenizes the 'summary' field of the input examples to prepare the target labels for the summarization task.
    # Sets a maximum token length of 128, and truncates any text longer than this limit.
    labels = tokenizer(text_target=examples["summary"], max_length=128, truncation=True)

    # Assigns the tokenized labels to the 'labels' field of model_inputs.
    # The 'labels' field is used during training to calculate the loss and guide model learning.
    model_inputs["labels"] = labels["input_ids"]

    # Returns the prepared inputs and labels as a single dictionary, ready for training.
    return model_inputs


In [ ]:
tokenized_billsum = billsum.map(preprocess_function, batched=True)

Map:   0%|          | 0/989 [00:00<?, ? examples/s]

Map:   0%|          | 0/248 [00:00<?, ? examples/s]

In [ ]:
tokenized_billsum['test'][0]['text']

'The people of the State of California do enact as follows:\n\n\nSECTION 1.\n(a) The Legislature finds and declares all of the following:\n(1) Short-lived climate pollutants, such as black carbon, fluorinated gases, and methane, are powerful climate forcers that have a dramatic and detrimental effect on air quality, public health, and climate change.\n(2) These pollutants create a warming influence on the climate that is many times more potent than that of carbon dioxide.\n(3) Short-lived climate pollutants that are toxic air contaminants also are a significant environmental risk factor for premature death.\n(4) Reducing emissions of these pollutants can have an immediate beneficial impact on climate change and on public health.\n(5) To the extent possible, efforts to reduce emissions of short-lived climate pollutants should focus on areas of the state that are disproportionately affected by poor air quality.\n(b) It is the intent of the Legislature to support the adoption of policies 

In [ ]:
tokenized_billsum['test'][0]['summary']

'(1)\xa0The California Global Warming Solutions Act of 2006 designates the State Air Resources Board as the state agency charged with monitoring and regulating sources of emissions of greenhouse gases. The state board is required to approve a statewide greenhouse gas emissions limit equivalent to the statewide greenhouse gas emissions level in 1990 to be achieved by 2020. The state board is also required to complete a comprehensive strategy to reduce emissions of short-lived climate pollutants, as defined, in the state.\nThis bill would require the state board, no later than January 1, 2018, to approve and begin implementing that comprehensive strategy to reduce emissions of short-lived climate pollutants to achieve a reduction in methane by 40%, hydrofluorocarbon gases by 40%, and anthropogenic black carbon by 50% below 2013 levels by 2030, as specified. The bill also would establish specified targets for reducing organic waste in landfills.\nThis bill would require the state board, i

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model="t5-small")

# Evaluation Metrics for Training

In [ ]:
rouge = evaluate.load("rouge")

In [ ]:
def compute_metrics(eval_pred):
    # Unpacks the evaluation predictions tuple into predictions and labels.
    predictions, labels = eval_pred

    # Decodes the tokenized predictions back to text, skipping any special tokens (e.g., padding tokens).
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)

    # Replaces any -100 values in labels with the tokenizer's pad_token_id.
    # This is done because -100 is often used to ignore certain tokens when calculating the loss during training.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)

    # Decodes the tokenized labels back to text, skipping any special tokens (e.g., padding tokens).
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Computes the ROUGE metric between the decoded predictions and decoded labels.
    # The use_stemmer parameter enables stemming, which reduces words to their root form before comparison.
    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    # Calculates the length of each prediction by counting the non-padding tokens.
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]

    # Computes the mean length of the predictions and adds it to the result dictionary under the key "gen_len".
    result["gen_len"] = np.mean(prediction_lens)

    # Rounds each value in the result dictionary to 4 decimal places for cleaner output, and returns the result.
    return {k: round(v, 4) for k, v in result.items()}

# Training the Model

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="my_fine_tuned_t5_small_model",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=5,
    predict_with_generate=True,
    fp16=True,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_billsum["train"],
    eval_dataset=tokenized_billsum["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,No log,2.623370,0.137000,0.046200,0.111700,0.111400,19.000000
2,No log,2.483130,0.153200,0.057500,0.125000,0.124900,19.000000
3,No log,2.427928,0.182900,0.082200,0.151400,0.151400,19.000000
4,No log,2.403649,0.188600,0.089600,0.157200,0.157000,19.000000
5,2.821400,2.394625,0.192000,0.092900,0.160900,0.160700,19.000000


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


TrainOutput(global_step=620, training_loss=2.772800765498992, metrics={'train_runtime': 407.8758, 'train_samples_per_second': 12.124, 'train_steps_per_second': 1.52, 'total_flos': 1338530416558080.0, 'train_loss': 2.772800765498992, 'epoch': 5.0})

In [ ]:
model_save_path = "my_fine_tuned_t5_small_model"

In [ ]:
trainer.save_model(model_save_path)

In [ ]:
tokenizer = T5Tokenizer.from_pretrained('t5-small')
tokenizer.save_pretrained(model_save_path)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


('my_fine_tuned_t5_small_model/tokenizer_config.json',
 'my_fine_tuned_t5_small_model/special_tokens_map.json',
 'my_fine_tuned_t5_small_model/spiece.model',
 'my_fine_tuned_t5_small_model/added_tokens.json')

# Use the Fine-Tuned Model to Summarize Text

In [ ]:
text = billsum['test'][100]['text']
text = "summarize: " + text
text

'summarize: The people of the State of California do enact as follows:\n\n\nSECTION 1.\nSection 11834.02 of the Health and Safety Code is amended to read:\n11834.02.\n(a)  As used in this chapter, “alcoholism or drug abuse recovery or treatment facility or facilities,” “facility,” or “facilities” means any premises, place, or building that provides 24-hour residential nonmedical services to adults who are recovering from problems related to alcohol, drug, or alcohol and drug misuse or abuse, and who need alcohol, drug, or alcohol and drug recovery treatment or detoxification services.\n(b)  As used in this chapter, “adults” may include, but is not limited to, all of the following:\n(1)  Mothers over 18 years of age and their children.\n(2)  Emancipated minors, which may include, but is not limited to, mothers under 18 years of age and their children.\n(c)  As used in this chapter, “emancipated minors” means persons under 18 years of age who have acquired emancipation status pursuant to

In [ ]:
summarizer = pipeline("summarization", model="my_fine_tuned_t5_small_model")
pred = summarizer(text)
pred


You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers
Token indices sequence length is longer than the specified maximum sequence length for this model (1944 > 512). Running this sequence through the model will result in indexing errors


[{'summary_text': 'Existing law prohibits a city, county, or other local public entity from placing restrictions on building heights, setback, lot dimensions, or placement of signs of an alcoholism or drug abuse recovery or treatment facility that serves six or fewer persons as long as the restrictions are identical to those applied to other single-family dwellings. This bill would require the department to issue a single license to the following types of alcoholisesm or drugs abuse recovery and treatment facilities, as defined, if the proposed location is in proximity to an existing facility, a boarding house, rooming house, or zoning ordinance that prohibits the use of a residential or a building that provides 24-hour residential nonmedical services to a person who is a business run for the care of minors or persons that is not required of an emancipation of the .'}]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("my_fine_tuned_t5_small_model")
inputs = tokenizer(text, return_tensors="pt").input_ids
inputs


Token indices sequence length is longer than the specified maximum sequence length for this model (1942 > 512). Running this sequence through the model will result in indexing errors


tensor([[21603,    10,    37,  ...,  3636,     5,     1]])

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained("my_fine_tuned_t5_small_model")
outputs = model.generate(inputs, max_new_tokens=100, do_sample=False)

In [ ]:
tokenizer.decode(outputs[0], skip_special_tokens=True)

'Existing law requires the department to approve a single license to the following types of alcoholism or drug abuse recovery or treatment facilities as defined. Existing law requires the department to approve a single license to the following types of alcoholism or drug abuse recovery or treatment facilities, as defined, if the proposed location is in proximity to an existing facility that would result in overconcentration. Existing law requires the department to approve a single license to the following types of alcoholis'

# Evaluate the Result

In [ ]:
pred[0]['summary_text']

'Existing law prohibits a city, county, or other local public entity from placing restrictions on building heights, setback, lot dimensions, or placement of signs of an alcoholism or drug abuse recovery or treatment facility that serves six or fewer persons as long as the restrictions are identical to those applied to other single-family dwellings. This bill would require the department to issue a single license to the following types of alcoholisesm or drugs abuse recovery and treatment facilities, as defined, if the proposed location is in proximity to an existing facility, a boarding house, rooming house, or zoning ordinance that prohibits the use of a residential or a building that provides 24-hour residential nonmedical services to a person who is a business run for the care of minors or persons that is not required of an emancipation of the .'

In [ ]:
preds = [pred[0]['summary_text']]

In [ ]:
labels = [billsum['test'][100]['summary']]

In [ ]:
rouge.compute(predictions=preds, references=labels, use_stemmer=True)

{'rouge1': 0.3059163059163059,
 'rouge2': 0.1649782923299566,
 'rougeL': 0.19336219336219337,
 'rougeLsum': 0.26551226551226553}

# Testing our own Paragraphs

In [ ]:
text1 = "Urbanization is a defining trend of the 21st century, with more than half of the world's population now living in cities. This shift presents both opportunities and challenges for sustainable development. On one hand, cities are engines of economic growth, innovation, and cultural exchange. They offer better access to jobs, education, healthcare, and social services, which can improve the quality of life for their residents. Urban areas are also more efficient in terms of resource use and can implement green technologies and infrastructure more effectively than rural areas. On the other hand, rapid urbanization often leads to significant challenges, including inadequate housing, congested transportation systems, and environmental degradation. Many cities struggle with air and water pollution, waste management, and the provision of essential services to all residents, particularly in informal settlements. Additionally, the high population density in urban areas can exacerbate social inequalities and lead to the marginalization of vulnerable groups. Addressing these issues requires integrated urban planning, investment in sustainable infrastructure, and inclusive policies that ensure all urban residents benefit from the opportunities cities offer. By fostering resilient, inclusive, and sustainable urban environments, we can harness the potential of cities to drive positive global change."
text2 = "The evolution of technology has dramatically transformed the way we communicate, work, and live, ushering in a new era of connectivity and convenience. With the advent of the internet, mobile devices, and social media platforms, information is now accessible at our fingertips, enabling instant communication and interaction across the globe. This technological advancement has revolutionized industries, from healthcare and education to entertainment and commerce, facilitating unprecedented innovation and efficiency. For instance, telemedicine allows patients to consult with doctors remotely, online learning platforms provide access to education for people worldwide, and e-commerce has transformed the retail landscape. However, this rapid technological progress also brings significant challenges. Issues such as digital addiction, cyberbullying, privacy concerns, and the spread of misinformation are becoming increasingly prevalent. Moreover, the digital divide persists, with many individuals and communities lacking access to the latest technologies. Balancing the benefits of technological advancement with its drawbacks requires thoughtful regulation, ethical considerations, and efforts to ensure that technological progress is inclusive and equitable."
text3 = "Education plays a pivotal role in shaping the future of individuals and societies, acting as a powerful catalyst for personal growth, social cohesion, and economic development. A robust education system equips students with critical thinking skills, knowledge, and the ability to adapt to rapidly changing circumstances. It fosters creativity, innovation, and problem-solving abilities, which are essential for addressing complex global challenges such as poverty, inequality, and environmental sustainability. Education also promotes social cohesion by instilling values of tolerance, empathy, and civic responsibility, contributing to more inclusive and peaceful societies. However, significant disparities in educational access and quality persist worldwide, with marginalized communities often facing barriers such as inadequate infrastructure, lack of trained teachers, and limited resources. These inequalities hinder the full realization of individuals' potential and impede societal progress. Ensuring inclusive and equitable education for all requires targeted policies, increased investment in educational infrastructure, teacher training, and community engagement. By addressing these challenges, we can unlock the full potential of every individual and pave the way for a more equitable and prosperous future."

In [ ]:
text = "summarize: " + text1
text

"summarize: Urbanization is a defining trend of the 21st century, with more than half of the world's population now living in cities. This shift presents both opportunities and challenges for sustainable development. On one hand, cities are engines of economic growth, innovation, and cultural exchange. They offer better access to jobs, education, healthcare, and social services, which can improve the quality of life for their residents. Urban areas are also more efficient in terms of resource use and can implement green technologies and infrastructure more effectively than rural areas. On the other hand, rapid urbanization often leads to significant challenges, including inadequate housing, congested transportation systems, and environmental degradation. Many cities struggle with air and water pollution, waste management, and the provision of essential services to all residents, particularly in informal settlements. Additionally, the high population density in urban areas can exacerbat

In [ ]:
summarizer = pipeline("summarization", model="my_fine_tuned_t5_small_model")
pred = summarizer(text)
pred

[{'summary_text': "urbanization is a defining trend of the 21st century, with more than half of the world's population now living in cities . This shift presents both opportunities and challenges for sustainable development . Cities are engines of economic growth, innovation, and cultural exchange ."}]

In [ ]:
text = "summarize: " + text2
text

'summarize: The evolution of technology has dramatically transformed the way we communicate, work, and live, ushering in a new era of connectivity and convenience. With the advent of the internet, mobile devices, and social media platforms, information is now accessible at our fingertips, enabling instant communication and interaction across the globe. This technological advancement has revolutionized industries, from healthcare and education to entertainment and commerce, facilitating unprecedented innovation and efficiency. For instance, telemedicine allows patients to consult with doctors remotely, online learning platforms provide access to education for people worldwide, and e-commerce has transformed the retail landscape. However, this rapid technological progress also brings significant challenges. Issues such as digital addiction, cyberbullying, privacy concerns, and the spread of misinformation are becoming increasingly prevalent. Moreover, the digital divide persists, with ma

In [ ]:
summarizer = pipeline("summarization", model="my_fine_tuned_t5_small_model")
pred = summarizer(text)
pred

[{'summary_text': 'telemedicine allows patients to consult with doctors remotely, online learning platforms provide access to education for people worldwide, and e-commerce has transformed the retail landscape. This rapid technological progress also brings significant challenges. Issues such as cyberbullying, privacy concerns, and the spread of misinformation are becoming increasingly prevalent.'}]

In [ ]:
text = "summarize: " + text3
text

"summarize: Education plays a pivotal role in shaping the future of individuals and societies, acting as a powerful catalyst for personal growth, social cohesion, and economic development. A robust education system equips students with critical thinking skills, knowledge, and the ability to adapt to rapidly changing circumstances. It fosters creativity, innovation, and problem-solving abilities, which are essential for addressing complex global challenges such as poverty, inequality, and environmental sustainability. Education also promotes social cohesion by instilling values of tolerance, empathy, and civic responsibility, contributing to more inclusive and peaceful societies. However, significant disparities in educational access and quality persist worldwide, with marginalized communities often facing barriers such as inadequate infrastructure, lack of trained teachers, and limited resources. These inequalities hinder the full realization of individuals' potential and impede societ

In [ ]:
summarizer = pipeline("summarization", model="my_fine_tuned_t5_small_model")
pred = summarizer(text)
pred

[{'summary_text': 'a robust education system equips students with critical thinking skills, knowledge, and the ability to adapt to rapidly changing circumstances. It fosters creativity, innovation, and problem-solving abilities, which are essential for addressing complex global challenges such as poverty, inequality, and environmental sustainability.'}]

# Tokenization Pre-Trained_T5

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("Falconsai/text_summarization")

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

In [ ]:
tokenized_text = tokenizer(example['text'])
for key in tokenized_text:
    print(key)
    print(tokenized_text[key])

Token indices sequence length is longer than the specified maximum sequence length for this model (1768 > 512). Running this sequence through the model will result in indexing errors


input_ids
[37, 151, 13, 8, 1015, 13, 1826, 103, 3, 35, 2708, 38, 6963, 10, 180, 3073, 9562, 1300, 100, 1810, 1522, 36, 801, 6, 11, 164, 36, 3, 11675, 6, 38, 8, 9896, 6859, 1983, 5, 180, 3073, 5, 1682, 37, 28204, 12902, 11, 15884, 7, 66, 13, 8, 826, 10, 41, 9, 61, 27223, 14652, 370, 502, 11, 70, 1791, 28, 3, 9, 1627, 1004, 21, 17711, 6, 2510, 6, 11, 694, 5, 3, 18536, 502, 1346, 383, 48, 1756, 19, 3510, 57, 1362, 11, 4879, 7137, 6, 1455, 18408, 6, 533, 3580, 6, 958, 688, 6, 11, 8, 5989, 2201, 681, 5, 41, 115, 61, 2150, 12, 321, 8, 2822, 1166, 7, 21, 14326, 4330, 11, 19715, 22, 7, 868, 1166, 21, 28905, 19715, 11, 4330, 11, 8, 1015, 1775, 13, 2575, 1685, 22, 7, 12741, 24382, 331, 6, 24614, 53, 19, 8, 1374, 1137, 13, 1687, 21, 1826, 502, 80, 12, 662, 203, 13, 1246, 6, 13066, 5, 41, 75, 61, 11180, 502, 5696, 1084, 18, 26, 3623, 29, 53, 15935, 11, 7905, 6, 68, 186, 13, 273, 502, 5696, 19598, 2660, 2317, 2241, 5157, 6, 84, 54, 991, 12, 280, 2961, 1036, 30307, 24, 1113, 59, 163, 8, 4161, 861, 1

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model="Falconsai/text_summarization")

# Load Model_Pre-Trained_T5 *(Text Summarization)*

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained("Falconsai/text_summarization")

config.json:   0%|          | 0.00/1.49k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="my_fine_tuned_t5_small_model_2",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,=-
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=5,
    predict_with_generate=True,
    fp16=True,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_billsum["train"],
    eval_dataset=tokenized_billsum["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,No log,2.693537,0.131800,0.043500,0.109200,0.108900,19.000000
2,No log,2.558708,0.148400,0.052500,0.119500,0.119400,19.000000
3,No log,2.505466,0.153500,0.055600,0.124700,0.124600,19.000000
4,No log,2.479368,0.155600,0.056100,0.126400,0.126300,19.000000
5,No log,2.470774,0.159200,0.059200,0.129900,0.129800,19.000000


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


TrainOutput(global_step=310, training_loss=2.7982947564894154, metrics={'train_runtime': 366.0291, 'train_samples_per_second': 13.51, 'train_steps_per_second': 0.847, 'total_flos': 1338530416558080.0, 'train_loss': 2.7982947564894154, 'epoch': 5.0})

In [ ]:
model_save_path = "my_fine_tuned_t5_small_model_2"

In [ ]:
trainer.save_model(model_save_path)

In [ ]:
tokenizer = T5Tokenizer.from_pretrained("Falconsai/text_summarization")
tokenizer.save_pretrained(model_save_path)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


('my_fine_tuned_t5_small_model_2/tokenizer_config.json',
 'my_fine_tuned_t5_small_model_2/special_tokens_map.json',
 'my_fine_tuned_t5_small_model_2/spiece.model',
 'my_fine_tuned_t5_small_model_2/added_tokens.json')

# Evaluating the Result

In [ ]:
text = billsum['test'][100]['text']
text = "summarize: " + text
text

'summarize: The people of the State of California do enact as follows:\n\n\nSECTION 1.\nSection 11834.02 of the Health and Safety Code is amended to read:\n11834.02.\n(a)  As used in this chapter, “alcoholism or drug abuse recovery or treatment facility or facilities,” “facility,” or “facilities” means any premises, place, or building that provides 24-hour residential nonmedical services to adults who are recovering from problems related to alcohol, drug, or alcohol and drug misuse or abuse, and who need alcohol, drug, or alcohol and drug recovery treatment or detoxification services.\n(b)  As used in this chapter, “adults” may include, but is not limited to, all of the following:\n(1)  Mothers over 18 years of age and their children.\n(2)  Emancipated minors, which may include, but is not limited to, mothers under 18 years of age and their children.\n(c)  As used in this chapter, “emancipated minors” means persons under 18 years of age who have acquired emancipation status pursuant to

In [ ]:
summarizer = pipeline("summarization", model="my_fine_tuned_t5_small_model_2")
pred = summarizer(text)
pred

Token indices sequence length is longer than the specified maximum sequence length for this model (1944 > 512). Running this sequence through the model will result in indexing errors


[{'summary_text': 'Existing law prohibits a city, county, or other local public entity from placing restrictions on building heights, setback, lot dimensions, or placement of signs of an alcoholism or drug abuse recovery or treatment facility that serves six or fewer persons, as long as the restrictions are identical to those applied to other single-family dwellings. Existing legislation prohibits the department from granting an application for a new facility license, if the proposed location is in proximity to an existing facility that would result in overconcentration of facilities.'}]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("my_fine_tuned_t5_small_model_2")
inputs = tokenizer(text, return_tensors="pt").input_ids
inputs

Token indices sequence length is longer than the specified maximum sequence length for this model (1942 > 512). Running this sequence through the model will result in indexing errors


tensor([[21603,    10,    37,  ...,  3636,     5,     1]])

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained("my_fine_tuned_t5_small_model_2")
outputs = model.generate(inputs, max_new_tokens=100, do_sample=False)

In [ ]:
tokenizer.decode(outputs[0], skip_special_tokens=True)

'Existing law requires the department to approve a single license to the following types of alcoholism or drug abuse recovery or treatment facilities as defined. Existing law requires the department to approve a single license to the following types of alcoholism or drug abuse recovery or treatment facilities, as defined, if the proposed location is in proximity to an existing facility that would result in overconcentration. Existing law requires the department to approve a single license to the following types of alcoholis'

In [ ]:
pred[0]['summary_text']

'Existing law prohibits a city, county, or other local public entity from placing restrictions on building heights, setback, lot dimensions, or placement of signs of an alcoholism or drug abuse recovery or treatment facility that serves six or fewer persons, as long as the restrictions are identical to those applied to other single-family dwellings. Existing legislation prohibits the department from granting an application for a new facility license, if the proposed location is in proximity to an existing facility that would result in overconcentration of facilities.'

In [ ]:
preds = [pred[0]['summary_text']]

In [ ]:
labels = [billsum['test'][100]['summary']]

In [ ]:
rouge.compute(predictions=preds, references=labels, use_stemmer=True)

{'rouge1': 0.20657276995305165,
 'rouge2': 0.1381475667189953,
 'rougeL': 0.1533646322378717,
 'rougeLsum': 0.18466353677621283}

# Testing on our own Paragraphs

In [ ]:
text1 = "Urbanization is a defining trend of the 21st century, with more than half of the world's population now living in cities. This shift presents both opportunities and challenges for sustainable development. On one hand, cities are engines of economic growth, innovation, and cultural exchange. They offer better access to jobs, education, healthcare, and social services, which can improve the quality of life for their residents. Urban areas are also more efficient in terms of resource use and can implement green technologies and infrastructure more effectively than rural areas. On the other hand, rapid urbanization often leads to significant challenges, including inadequate housing, congested transportation systems, and environmental degradation. Many cities struggle with air and water pollution, waste management, and the provision of essential services to all residents, particularly in informal settlements. Additionally, the high population density in urban areas can exacerbate social inequalities and lead to the marginalization of vulnerable groups. Addressing these issues requires integrated urban planning, investment in sustainable infrastructure, and inclusive policies that ensure all urban residents benefit from the opportunities cities offer. By fostering resilient, inclusive, and sustainable urban environments, we can harness the potential of cities to drive positive global change."
text2 = "The evolution of technology has dramatically transformed the way we communicate, work, and live, ushering in a new era of connectivity and convenience. With the advent of the internet, mobile devices, and social media platforms, information is now accessible at our fingertips, enabling instant communication and interaction across the globe. This technological advancement has revolutionized industries, from healthcare and education to entertainment and commerce, facilitating unprecedented innovation and efficiency. For instance, telemedicine allows patients to consult with doctors remotely, online learning platforms provide access to education for people worldwide, and e-commerce has transformed the retail landscape. However, this rapid technological progress also brings significant challenges. Issues such as digital addiction, cyberbullying, privacy concerns, and the spread of misinformation are becoming increasingly prevalent. Moreover, the digital divide persists, with many individuals and communities lacking access to the latest technologies. Balancing the benefits of technological advancement with its drawbacks requires thoughtful regulation, ethical considerations, and efforts to ensure that technological progress is inclusive and equitable."
text3 = "Education plays a pivotal role in shaping the future of individuals and societies, acting as a powerful catalyst for personal growth, social cohesion, and economic development. A robust education system equips students with critical thinking skills, knowledge, and the ability to adapt to rapidly changing circumstances. It fosters creativity, innovation, and problem-solving abilities, which are essential for addressing complex global challenges such as poverty, inequality, and environmental sustainability. Education also promotes social cohesion by instilling values of tolerance, empathy, and civic responsibility, contributing to more inclusive and peaceful societies. However, significant disparities in educational access and quality persist worldwide, with marginalized communities often facing barriers such as inadequate infrastructure, lack of trained teachers, and limited resources. These inequalities hinder the full realization of individuals' potential and impede societal progress. Ensuring inclusive and equitable education for all requires targeted policies, increased investment in educational infrastructure, teacher training, and community engagement. By addressing these challenges, we can unlock the full potential of every individual and pave the way for a more equitable and prosperous future."

In [ ]:
text = "summarize: " + text1
text

"summarize: Urbanization is a defining trend of the 21st century, with more than half of the world's population now living in cities. This shift presents both opportunities and challenges for sustainable development. On one hand, cities are engines of economic growth, innovation, and cultural exchange. They offer better access to jobs, education, healthcare, and social services, which can improve the quality of life for their residents. Urban areas are also more efficient in terms of resource use and can implement green technologies and infrastructure more effectively than rural areas. On the other hand, rapid urbanization often leads to significant challenges, including inadequate housing, congested transportation systems, and environmental degradation. Many cities struggle with air and water pollution, waste management, and the provision of essential services to all residents, particularly in informal settlements. Additionally, the high population density in urban areas can exacerbat

In [ ]:
summarizer = pipeline("summarization", model="my_fine_tuned_t5_small_model_2")
pred = summarizer(text)
pred

[{'summary_text': "Urbanization is a defining trend of the 21st century, with more than half of the world's population now living in cities . This shift presents both opportunities and challenges for sustainable development. Cities are engines of economic growth, innovation, and cultural exchange. They offer better access to jobs, education, healthcare, and social services, which can improve the quality of life for their residents."}]

In [ ]:
text = "summarize: " + text2
text

'summarize: The evolution of technology has dramatically transformed the way we communicate, work, and live, ushering in a new era of connectivity and convenience. With the advent of the internet, mobile devices, and social media platforms, information is now accessible at our fingertips, enabling instant communication and interaction across the globe. This technological advancement has revolutionized industries, from healthcare and education to entertainment and commerce, facilitating unprecedented innovation and efficiency. For instance, telemedicine allows patients to consult with doctors remotely, online learning platforms provide access to education for people worldwide, and e-commerce has transformed the retail landscape. However, this rapid technological progress also brings significant challenges. Issues such as digital addiction, cyberbullying, privacy concerns, and the spread of misinformation are becoming increasingly prevalent. Moreover, the digital divide persists, with ma

In [ ]:
summarizer = pipeline("summarization", model="my_fine_tuned_t5_small_model_2")
pred = summarizer(text)
pred

[{'summary_text': 'Information is now accessible at our fingertips, enabling instant communication and interaction across the globe, ushering in a new era of connectivity and convenience. This technological advancement has revolutionized industries, from healthcare and education to entertainment and commerce, facilitating unprecedented innovation and efficiency . Issues such as cyberbullying, privacy concerns, and the spread of misinformation are becoming increasingly prevalent.'}]

In [ ]:
text = "summarize: " + text3
text

"summarize: Education plays a pivotal role in shaping the future of individuals and societies, acting as a powerful catalyst for personal growth, social cohesion, and economic development. A robust education system equips students with critical thinking skills, knowledge, and the ability to adapt to rapidly changing circumstances. It fosters creativity, innovation, and problem-solving abilities, which are essential for addressing complex global challenges such as poverty, inequality, and environmental sustainability. Education also promotes social cohesion by instilling values of tolerance, empathy, and civic responsibility, contributing to more inclusive and peaceful societies. However, significant disparities in educational access and quality persist worldwide, with marginalized communities often facing barriers such as inadequate infrastructure, lack of trained teachers, and limited resources. These inequalities hinder the full realization of individuals' potential and impede societ

In [ ]:
summarizer = pipeline("summarization", model="my_fine_tuned_t5_small_model_2")
pred = summarizer(text)
pred

[{'summary_text': 'A robust education system equips students with critical thinking skills, knowledge, and the ability to adapt to rapidly changing circumstances. It fosters creativity, innovation, and problem-solving abilities, which are essential for addressing complex global challenges such as poverty, inequality, and environmental sustainability . Education also promotes social cohesion by instilling values of tolerance, empathy, and civic responsibility, contributing to more inclusive and peaceful societies.'}]

In [ ]:
text4 = "python a high-level interpreted programming language famous for its zen-like code it's arguably the most popular language in the world because it's easy to learn yet practical for serious projects in fact you're watching this youtube video in a python web application right now it was created by guido van rossum and released in 1991 who named it after monty python's flying circus which is why you'll sometimes find spam and eggs instead of foo and bar in code samples it's commonly used to build server-side applications like web apps with the django framework and is the language of choice for big data analysis and machine learning many students choose python to start learning to code because of its emphasis on readability as outlined by the zen of python beautiful is better than ugly while explicit is better than implicit python is very simple but avoids the temptation to sprinkle in magic that causes ambiguity its code is often organized into notebooks where individual cells can be executed then documented in the same place we're currently at version 3 of the language and you can get started by creating a file that ends in py or ipymb to create an interactive notebook create a variable by setting a name equal to a value it's strongly typed which means values won't change in unexpected ways but dynamic so type annotations are not required the syntax is highly efficient allowing you to declare multiple variables on a single line and define tuples lists and dictionaries with a literal syntax semicolons are not required and if you use them and experience pythonista will say that your code is not pythonic instead of semicolons python uses indentation to terminate or determine the scope of a line of code define a function with the def keyword then indent the next line usually by four spaces to define the function body we might then add a for loop to it and indent that by another four spaces this eliminates the need for curly braces and semicolons found in many other languages python is a multi-paradigm language we can apply functional programming patterns with things like anonymous functions using lambda it also uses objects as an abstraction for data allowing you to implement object-oriented patterns with things like classes and inheritance it also has a huge ecosystem of third-party libraries such as deep learning frameworks like tensorflow and wrappers for many high performance low level packages like open computer vision which are most often installed with the pip package manager this has been the python programming language in 100 seconds hit the like button if you want to see more short videos like this thanks for watching and i will see you in the next one"
text = "summarize: " + text4
text

"summarize: python a high-level interpreted programming language famous for its zen-like code it's arguably the most popular language in the world because it's easy to learn yet practical for serious projects in fact you're watching this youtube video in a python web application right now it was created by guido van rossum and released in 1991 who named it after monty python's flying circus which is why you'll sometimes find spam and eggs instead of foo and bar in code samples it's commonly used to build server-side applications like web apps with the django framework and is the language of choice for big data analysis and machine learning many students choose python to start learning to code because of its emphasis on readability as outlined by the zen of python beautiful is better than ugly while explicit is better than implicit python is very simple but avoids the temptation to sprinkle in magic that causes ambiguity its code is often organized into notebooks where individual cells 

In [ ]:
summarizer = pipeline("summarization", model="my_fine_tuned_t5_small_model_2")
pred = summarizer(text)
pred

Token indices sequence length is longer than the specified maximum sequence length for this model (620 > 512). Running this sequence through the model will result in indexing errors


[{'summary_text': 'python is a multi-paradigm language we can apply functional programming patterns with things like anonymous functions using lambda it also uses objects as an abstraction for data allowing you to implement object-oriented patterns with something like classes and inheritance it also has a huge ecosystem of third-party libraries such as deep learning frameworks like tensorflow and wrappers for many high performance low level packages like open computer vision .'}]